# Training a speech-to-text model using Huggingsound

<a target="_blank" href="https://colab.research.google.com/github/Koffair/colab_pipelines/blob/main/notebooks/03_train_s2t.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**WARNING**
- Open the notebook in Colab by clicking on the "Open in Colab" badge at the top of the notebook
- Save a copy of the notebook to your Google Drive by clicking "File" > "Save a Copy to Drive"
- This notebook assumes that you have downloaded the necessery data and preprocessed it by following the instructions in ```01_train_language_models.ipynb```


## Setup

In [1]:
!pip install huggingsound

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.4/776.4 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import os

import pandas as pd
from huggingsound import (
    ModelArguments,
    SpeechRecognitionModel,
    TokenSet,
    TrainingArguments,
)
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Preprocessing data

In [17]:
clip_prefix = "/content/gdrive/My Drive/Colab Notebooks/corpora/common_voice/2023.03.15/cv-corpus-13.0-2023-03-09/hu/clips"

df = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/corpora/common_voice/2023.03.15/cv-corpus-13.0-2023-03-09/hu/validated.tsv", sep="\t")
df = df[df["down_votes"] == 0]  # use only validated data without down_votes
clips = df["path"]
clips = [e for e in clips if os.path.isfile(os.path.join(clip_prefix, e)) and os.stat(os.path.join(clip_prefix, e)).st_size != 0]
df = df[df["path"].isin(clips)]

print(df.shape)

(28840, 11)


In [18]:
train, test = train_test_split(df, test_size=0.2)

In [19]:
trainx = zip(train["path"], train["sentence"])
testx = zip(test["path"], test["sentence"])


def clean_sentence(sentence):
    wds = word_tokenize(sentence)
    return " ".join([wd.lower() for wd in wds if wd.isalnum()])


train_data = [
    {"path": os.path.join(clip_prefix, e[0]), "transcription": clean_sentence(e[1])}
    for e in trainx
]
eval_data = [
    {"path": os.path.join(clip_prefix, e[0]), "transcription": clean_sentence(e[1])}
    for e in testx
]

## Model setup

In [20]:
model = SpeechRecognitionModel("facebook/wav2vec2-large-xlsr-53")
output_dir = "/content/gdrive/My Drive/Colab Notebooks/models/s2t"

tokens = [
    "a",
    "á",
    "b",
    "c",
    "d",
    "e",
    "é",
    "f",
    "g",
    "h",
    "i",
    "í",
    "j",
    "k",
    "l",
    "m",
    "n",
    "o",
    "ó",
    "ö",
    "ő",
    "p",
    "q",
    "r",
    "s",
    "t",
    "u",
    "ú",
    "ü",
    "ű" "v",
    "w",
    "x",
    "y",
    "z",
]
token_set = TokenSet(tokens)

INFO:huggingsound.speech_recognition.model:Loading model...
Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'project_hid.bias', 'quantizer.weight_proj.bias', 'project_hid.weight', 'project_q.weight', 'project_q.bias', 'quantizer.weight_proj.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should pro

## Fine-tune model

In [ ]:
model.finetune(
    output_dir,
    train_data=train_data,
    eval_data=eval_data,
    token_set=token_set,
)

`use_fast` is set to `True` but the tokenizer class does not have a fast version.  Falling back to the slow version.
INFO:huggingsound.speech_recognition.model:Loading training data...
INFO:huggingsound.speech_recognition.model:Converting data format...
INFO:huggingsound.speech_recognition.model:Preparing data input and labels...


Map:   0%|          | 0/23072 [00:00<?, ? examples/s]

In [16]:
!ls '/content/gdrive/My Drive/Colab Notebooks/corpora/common_voice/2023.03.15/cv-corpus-13.0-2023-03-09/hu/clips/' | wc -l

56246
